In [3]:
%load_ext autoreload
%load_ext line_profiler
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [78]:
import compmod2 as cp2
import argiope as ag
import pandas as pd
import numpy as np
from string import Template
import inspect, os, local_settings



In [74]:
# USEFUL FUNCTIONS
def create_dir(path):
    try:
        os.mkdir(path)
    except:
        pass
# SETTINGS
workdir   = "_workdir/"
outputdir = "outputs/"
label   = "indentation_2D"

create_dir(workdir)
create_dir(workdir + outputdir)


In [82]:
shape = np.array([4, 4, 4])
Ne = shape.prod()

def element_map(mesh):
    mesh.elements.loc[:, ("type", "solver", "") ] = "C3D8"
    return mesh
    
    
def material_map(mesh):
    mesh.elements.materials = ["mat{0}".format(i) for i in mesh.elements.index]
    return mesh

dist = compmod2.distributions.Weibull(k = 1., l = 1.e-2)
xt, x = dist.discretize(Ne, xmax = 1.)
np.random.shuffle(x)
materials = [ag.materials.ElasticPerfectlyPlastic(
                                 label = "mat{0}".format(i+1), 
                                 young_modulus = E, 
                                 poisson_ratio = nu, 
                                 yield_stress = x[i] * E) 
                                 for i in range(Ne)]

steps = [cp2.models.RVEStep(name = "loading", 
                            cx = ("disp", .1),
                            field_output_frequency = 1),
         cp2.models.RVEStep(name = "loading2", 
                            cx = ("disp", .1),
                            cy = ("disp", .1),
                            field_output_frequency = 1),]

sample = cp2.models.RVESample(shape = shape,
                              element_map = element_map,
                              material_map = material_map)
model = cp2.models.RVETest(label = "RVE",
                           parts = {"sample":sample},
                           steps = steps,
                           materials = materials,
                           solver = "abaqus", 
                           solver_path = local_settings.ABAQUS_PATH,
                           workdir = workdir,
                           verbose = True)
model.write_input()
model.run_simulation()

<Running "RVE" using abaqus>
(b'Abaqus JOB RVE\nAbaqus 6.13-1\nBegin Analysis Input File Processor\nTue Feb 20 22:57:24 2018\nRun pre\nTue Feb 20 22:57:26 2018\nEnd Analysis Input File Processor\nBegin Abaqus/Standard Analysis\nTue Feb 20 22:57:26 2018\nRun standard\nTue Feb 20 22:57:37 2018\nEnd Abaqus/Standard Analysis\nAbaqus JOB RVE COMPLETED\n', None)
<Ran RVE: duration 17.12s>


In [ ]:
mesh = argiope.mesh.structured_mesh(shape = (10, 10, 10))
mesh.nodes[("sets", "truc")] = mesh.nodes.coords.x == 0.
%timeit mesh.write_inp()

In [ ]:
#%timeit s = materials[0].write_inp() + "\n"
print(materials[0].write_inp())

In [ ]:
%%time 
out = ""
for mat in materials:
    out += mat.write_inp() + "\n"


In [ ]:
%%time 
s = "\n".join([mat.write_inp() for mat in materials])


In [ ]:
def list_to_string(l = range(200), width = 40, indent = "  "):
    """
    Converts a list-like to string with given line width.
    """
    l = [str(v) + "," for v in l]
    counter = 0
    out = "" + indent
    for w in l:
        s = len(w)
        if counter + s > width: 
            out += "\n" + indent
            counter = 0
        out += w
        counter += s
    return out.strip(",")


print(list_to_string())

In [20]:
print(step_template)

********************************************************************************
** STEPS: $NAME
********************************************************************************
*STEP, NAME = $NAME, NLGEOM = YES, INC = 1000000
*STATIC
$FRAMEDURATION, $DURATION, $MINFRAMEDURATION, $FRAMEDURATION
** BOUNDARY CONDITIONS
*BOUNDARY, OP = NEW
$DISP_CONTROL
iSample.pinned, 1, 3
** LOADS
$LOAD_CONTROL
*RESTART, WRITE, FREQUENCY = 0
*OUTPUT, FIELD, FREQUENCY = $FIELD_OUTPUT_FREQUENCY
*NODE OUTPUT
U
*ELEMENT OUTPUT
EVOL, E, LE, PE, EE, PEEQ, S
*OUTPUT, HISTORY
*ENERGY OUTPUT
ALLPD, ALLSE, ALLWK
*NODE OUTPUT, NSET = iSample.control
RF, CF, U
*End Step



In [ ]:
m.nodes.loc[1000, [("coords")]] = 10,10,10
m.nodes.loc[1000,"sets"] = False
m.nodes[("sets", "controlled")] = False

In [ ]:

n.coords[n.sets.left]


In [ ]:
n.coords[n.sets.left  ].sort_values(["y", "z"],ascending = False)

In [ ]:
n.coords[n.sets.right].sort_values(["y", "z"], ascending = False)

In [ ]:
dist = compmod2.distributions.Weibull(k = 5., l = .01)
xt, x = dist.discretize(Ne, xmax = 1. )

In [ ]:
import compmod2.distributions

In [ ]:
eq = argiope.utils._equation
%timeit eq(nodes = ["uright", "uleft", "control"], dofs = [1,1,1], coefficients = [1., -1., -1.])


In [ ]:
%timeit s = sample.mesh.write_inp()

In [ ]:
mesh = argiope.mesh.structured_mesh(shape = (10, 10, 10))
for i in range(10):
    mesh.nodes[("sets", "truc{0}".format(i))] = mesh.nodes.coords.x == 0.
mesh.elements.materials = ["mat{0}".format(i) for i in mesh.elements.index]
mesh.elements.loc[:, ("type", "solver", "") ] = "C3D8"
%timeit s=mesh.write_inp()

In [ ]:
m.elements.head()

In [ ]:
def fstring():
    out = ""
    for i in range(1000):
        out += "aaaa\n"
    return out

%timeit fstring()

In [ ]:
import numba
nbfstring = numba.jit(fstring)
%timeit nbfstring()

In [48]:
a = np.arange(10)
hasattr(a, "shape")

True